# E KNN and K-means CIFAR
_5 points_

transform cifar-10 to grayscale. 

- Does knn work similarly good?
- Does k-means work similarly good?
- Demonstrate this similar to  B, C and D

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

plt.set_cmap(None)

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# CIFAR-10 Dataset

https://www.cs.toronto.edu/~kriz/cifar.html

## Download and prepare Data

In [2]:
cifar_train = tf.keras.datasets.cifar10.load_data()[0]
cifar_test = tf.keras.datasets.cifar10.load_data()[1]

train_image, train_label = cifar_train[0], cifar_train[1]
test_image, test_label = cifar_test[0], cifar_test[1]






## Turn pictures to grayscale

In [3]:
# Auxiliary function for turning an RGB image to Gray-scale image
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

train_image = rgb2gray(train_image)
test_image = rgb2gray(test_image)
plt.set_cmap('gray')


## Reshaping

In [4]:
train_size = 100
test_size = 1000

train_image = train_image[:train_size]
train_label = train_label[:train_size]

test_image = test_image[:test_size]
test_label = test_label[:test_size]

n_train = train_image.shape[0]
n_test = test_image.shape[0]

m = train_image.shape[1]**2

print(train_image.shape)
print(test_image.shape)

train_image = train_image.reshape([n_train, m])
test_image = test_image.reshape([n_test, m])

(100, 32, 32)
(1000, 32, 32)


## Algorithm

## KMeans

Wir clustern nur Test Sample, für schnellere Laufzeiten.

In [29]:
from sklearn.cluster import KMeans


### Analyis of the clusters (Similar to C)

In [33]:
from sklearn.metrics.cluster import homogeneity_score
#flatten array to have one dimension less
#for correct format in homogeneity score method
test_label = test_label.flatten()

def doKmeans(iter):
  n_digits = len(np.unique(test_label))

  kmeans = KMeans(init='k-means++', n_clusters=n_digits, n_init=10, n_jobs=-1, verbose=0, max_iter=iter)

  X_test_local = test_image.reshape([n_test, m])
  #print(X_test_local.shape)
  #print(Y_test.shape)

  kmeans.fit(X_test_local)

  for y in range(10):
      print("\nCluster: %d" % (y))
      cluster = (np.array([i for i, x in enumerate(kmeans.labels_) if x == y]))
      labels_cluster_1 = test_label[cluster]
      len_cluster = len(labels_cluster_1)

      unique, counts = np.unique(labels_cluster_1, return_counts=True)
      counts_dict = dict(zip(unique, counts))
      majority_class = max(counts_dict.items(), key=lambda k: k[1])[0]

      print(counts_dict)
      print("Percentage of majority class: %f" % (max(counts)/float(len_cluster)))
      print("Majority class is: %d" % (majority_class))


      print("Laenge: ", len_cluster)
      print("Homogeneity Score for %d: %f" %( iter, homogeneity_score(test_label, kmeans.labels_)))
      

In [34]:
iter = [10,100,1000]
for x in range(len(iter)):
  print()
  print()
  print("________________________________________________________________________________")
  print("Anzahl Iterations %d" %iter[x])
  print("________________________________________________________________________________")
  
  doKmeans(iter[x])
  # have some whitespacese
  print()
  print()
  print()
  print("________________________________________________________________________________")
  print("________________________________________________________________________________")





________________________________________________________________________________
Anzahl Iterations 10
________________________________________________________________________________

Cluster: 0
{0: 5, 1: 12, 2: 12, 3: 11, 4: 6, 5: 6, 6: 20, 7: 6, 8: 9, 9: 10}
Percentage of majority class: 0.206186
Majority class is: 6
Laenge:  97
Homogeneity Score for 10: 0.094664

Cluster: 1
{0: 6, 1: 4, 2: 7, 3: 3, 4: 4, 5: 5, 6: 2, 7: 6, 8: 4, 9: 6}
Percentage of majority class: 0.148936
Majority class is: 2
Laenge:  47
Homogeneity Score for 10: 0.094664

Cluster: 2
{0: 10, 1: 1, 2: 3, 4: 1, 5: 4, 6: 2, 7: 2, 8: 4}
Percentage of majority class: 0.370370
Majority class is: 0
Laenge:  27
Homogeneity Score for 10: 0.094664

Cluster: 3
{0: 9, 1: 3, 3: 1, 4: 2, 6: 1, 7: 2, 8: 7, 9: 19}
Percentage of majority class: 0.431818
Majority class is: 9
Laenge:  44
Homogeneity Score for 10: 0.094664

Cluster: 4
{0: 2, 1: 4, 2: 4, 3: 4, 4: 5, 6: 8, 7: 3, 9: 1}
Percentage of majority class: 0.258065
Majority cla

### Gedanken zu Kmeans mit Cifar
- der Algorithmus funktioniert hier sehr schlecht. Es wurde so gut wie nicht geclustert. Das können wir an der Homogenität von 0,09 erkennen. 
- Bei Mnist haben wir eine Homogenität von über 0,5

## KNN

### Execute Black Magic

In [5]:
from sklearn.metrics import classification_report #needed for differentiating accuracy for each digit
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

neigh = KNeighborsClassifier(n_neighbors=8, n_jobs=-1)#since we have to try it with k = 2,4,6,8
neigh.fit(train_image, train_label)
pred = neigh.predict(test_image)

/usr/local/envs/py3env/lib/python3.5/site-packages/ipykernel/__main__.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [6]:
print("Score: %f" % neigh.score(test_image, test_label))
from sklearn.metrics import classification_report #needed for differentiating accuracy for each digit

classification_report = classification_report(test_label, pred)

print(classification_report)

Score: 0.135000
             precision    recall  f1-score   support

          0       0.08      0.02      0.03       103
          1       0.17      0.04      0.07        89
          2       0.10      0.29      0.15       100
          3       0.17      0.24      0.20       103
          4       0.11      0.46      0.18        90
          5       0.00      0.00      0.00        86
          6       0.00      0.00      0.00       112
          7       0.17      0.01      0.02       102
          8       0.33      0.04      0.07       106
          9       0.26      0.27      0.26       109

avg / total       0.14      0.14      0.10      1000



## analysis like in B with different Ks